In [ ]:
# %pip install pinecone-client

In [ ]:
# %pip install cohere

In [2]:
import os

from openai import OpenAI
client = OpenAI()

from pinecone import Pinecone
from pinecone import PodSpec

d:\Git Hub Repos\EcoFactory\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
use_serverless = "false"

# initialize connection to pinecone
# api_key = os.environ.get('PINECONE_API_KEY')
# print(api_key)
# configure client
pc = Pinecone(api_key="c511f2f3-19d6-458d-84b3-5336b57d7c80")      

In [4]:
# Index specification
spec = PodSpec(environment="gcp-starter")

In [6]:
# Giving our index a name
index_name = "eco-factor"

In [7]:
# # Delete the index, if an index of the same name already exists
# if index_name in pc.list_indexes().names():
#     pc.delete_index(index_name)

In [11]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=768,
        metric='cosine',
        spec=spec
    )

# connect to index
index = pc.Index(index_name)

In [12]:

from dotenv import load_dotenv
load_dotenv()
cohere_api_key = os.getenv('COHERE_API_KEY')
print(cohere_api_key)


lePj6yC6ouKnUeprmrKtzd7R9uHOsfl2t5kSqd3G


In [14]:
import cohere
co = cohere.Client(cohere_api_key)

In [40]:
descriptions = []
folder_path = 'D:/Git Hub Repos/EcoFactory/server/Notebook/product datasets'
files = os.listdir(folder_path)
for file in files:
    file_name = file
    if file_name.endswith('.txt'):
        with open(os.path.join(folder_path, file_name), 'r', encoding='utf-8') as file:
            text = file.read()
            descriptions.append(text)




['**[ANSWER 001]**\n\n"# 1- Conventional Oil Desalter\n\n## "Essential purification, proven efficiency."\n\n### Introduction to the Product:\nThe Conventional Oil Desalter is a critical component in refinery operations, responsible for removing salt and other impurities from crude oil before the distillation process. This technology has been a pillar in refineries, ensuring the quality of the processed oil and protecting downstream units from corrosion and deposits.\n\n# 2- Main Features and Product Indication\n\n### General Features:\nThis system is essential for refineries processing crude oil of different qualities, especially those with high salinity levels. Using electrostatic principles to coalesce and remove saltwater droplets from the oil, conventional desalters are designed to operate effectively, ensuring the integrity and efficiency of subsequent refining processes.\n\n# 3- Technical Product Information\n\n- **Processing Capacity:** Varies according to the model, but commonl

In [ ]:
print(descriptions[2])

In [59]:
# vectors = []
ids = ['Conventional Oil Desalter','Cyclone Pro-Elite Helmet', 'Sneaker Pro X100']
for id, description in zip(ids, descriptions):
    description = [description]
    response = co.embed(texts=description, model='multilingual-22-12').embeddings
    print(response)
    # vectors.append(response)
    index.upsert(vectors=zip(id, response))
   
    
    

[[-0.039276123, -0.091308594, 0.21606445, 0.2590332, 0.21252441, 0.11279297, 0.34277344, -0.26367188, 0.0309906, 0.42163086, -0.50927734, -0.31176758, -0.28027344, -0.10070801, 0.23864746, 0.022369385, 0.4958496, 0.45898438, -0.24804688, 0.27490234, 0.120666504, 0.60498047, 0.45263672, -0.039978027, -0.13000488, -0.18017578, -0.5385742, 0.10430908, -0.007926941, 0.23034668, 0.17602539, -0.25097656, -0.12561035, 0.47509766, -1.1103516, 0.42578125, -0.22216797, 0.2902832, -0.44970703, 0.34033203, -0.1697998, 0.017211914, -0.22766113, 0.35888672, -0.0029735565, -0.49536133, 0.29638672, 0.23352051, 0.42919922, -0.2548828, -0.22961426, 0.021514893, -0.36816406, 0.093933105, -0.56640625, 0.41210938, -0.07110596, 0.8886719, 0.16625977, -0.0016212463, -0.18286133, 0.2644043, -0.05218506, 0.0052375793, 0.48754883, 0.15197754, -0.093566895, 0.20031738, 0.124572754, 0.21948242, 0.19177246, -0.07446289, 0.15393066, 0.26782227, 0.13586426, 0.014312744, 0.27612305, -0.49975586, 0.19042969, -0.026992

In [60]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 2e-05,
 'namespaces': {'': {'vector_count': 2}},
 'total_vector_count': 2}

In [ ]:
# %pip install pandas

In [53]:
# import pandas as pd
# df = pd.DataFrame(
#     data = {
#     'id': ['Conventional Oil Desalter','Cyclone Pro-Elite Helmet', 'Sneaker Pro X100'],
#     'extracted_text': [vectors[0], vectors[1], vectors[2]]
#     })
# df

,id,extracted_text
0,Conventional Oil Desalter,"[[-0.039276123, -0.091308594, 0.21606445, 0.25..."
1,Cyclone Pro-Elite Helmet,"[[0.2010498, 0.23669434, 0.7285156, -0.2198486..."
2,Sneaker Pro X100,"[[0.36938477, 0.22924805, 0.23120117, -0.03631..."


In [ ]:
# %pip install numpy

In [31]:
# import numpy as np
# embeds = np.array(response)
# print(embeds.shape)

(1, 768)


Query the database

In [66]:
query = "General Features:Ideal for professional cyclists and serious cycling enthusiasts, the Cyclone Pro-Elite Helmet offers innovative features such as precise fit, optimized ventilation, and superior aerodynamics. This helmet is the perfect choice for those looking to improve their performance while maintaining safety at all speed levels."

# create the query embedding
vec = co.embed(
    texts=[query],
    model='multilingual-22-12',
    truncate='NONE'
).embeddings

# print(np.array(xq).shape)

# query, returning the top 10 most similar results
res = index.query(vector=vec, top_k=10, include_metadata=True)
res


# ans['matches'][0]['values']



{'matches': [{'id': 'C', 'score': 0.962923527, 'values': []},
             {'id': 'S', 'score': 0.810164571, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

In [68]:
for match in res['matches']:
    print(f"{match['score']:.2f}:")
    print(f"{match['score']:.2f}:")
    print(f"{match['score']:.2f}:")

0.96:
0.96:
0.96:
0.81:
0.81:
0.81:


In [ ]:
limit = 1600

def retrieve(query):
    xq = co.embed(
        texts=[query],
        model='multilingual-22-12',
        truncate='NONE'
    ).embeddings
    # search pinecone index for context
    xc = index.query(vector=xq, top_k=3, include_metadata=True)

    # Extract relevant information from the matches
    surahs = [str(x['metadata']['Surah']) for x in xc['matches']]
    ayahs = [str(x['metadata']['Ayat']) for x in xc['matches']]
    arabics = [str(x['metadata']['Arabic']) for x in xc['matches']]
    tafaseers = [str(x['metadata']['Tafaseer1']) for x in xc['matches']]

    # Combine the information into formatted contexts
    contexts = [
        f"Surah: {surah}\n Ayah: {ayah}\n Arabic: {arabic}\n"
        for surah, ayah, arabic in zip(surahs, ayahs, arabics)
    ]

    # Build the prompt with the retrieved contexts included
    prompt_start = (
        f"Answer the Query based on the contexts, if it's not in the contexts say 'I don't know the answer'. \n\n"
        f"Context:\n"
    )
    prompt_end = (
        f"\n\nQuery: {query}\nAnswer in the language of Query, if Query is in English Answer in English. Please provide reference Quran verses."
    )

    # Append contexts until hitting the limit
    for i in range(1, len(contexts)):
        if len("".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "".join(contexts) +
                prompt_end
            )
    return prompt

# Example usage with a query in a different language (Spanish)
query_new = "¿Cuál es el significado de la vida?"
prompt_new = retrieve(query_new)
print(prompt_new)

In [ ]:
def complete(prompt):
  response = co.generate(
                          model='c4ai-aya',
                          prompt=prompt,
                          max_tokens=600,
                          temperature=0.2,
                          k=0,
                          stop_sequences=['\n\n'],
                          return_likelihoods='NONE'
                        )
  return response.generations[0].text.strip()

In [ ]:
complete(prompt_new)